# **LINEARISATION UNIQUE DES TRAFICS SUR BORDEAUX METROPOLE**
> Traitements à mettre en oeuvre

In [83]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import Connexion_Transfert as ct
import geopandas as gp
import pandas as pd
import numpy as np
import re
from difflib import get_close_matches,SequenceMatcher
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from collections import Counter
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. MISE EN FORME DES COMPTAGES PONCTUELS

In [89]:
#import des données
cpt_pct=pd.read_excel(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_source\Bdx-Metro\comptages\base_comptages_routiers_v4.xlsx')
cpt_pct.columns=[a.lower() for a in cpt_pct.columns]
cpt_pct = gp.GeoDataFrame(cpt_pct, geometry=gp.points_from_xy(cpt_pct.longitude, cpt_pct.latitude))
cpt_pct.crs = {'init' :'epsg:4326'}
cpt_pct_l93=cpt_pct.to_crs({'init': 'epsg:2154'})
cpt_pct_l93['x_l93']=cpt_pct_l93.geometry.apply(lambda x : x.x)
cpt_pct_l93['y_l93']=cpt_pct_l93.geometry.apply(lambda x : x.y)
#mise en forme
cpt_pct_l93['nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : re.sub(('é|è|ê'),'e',x.lower()))

In [90]:
#cluster à 200m
data_test_clust=[[x, y] for x, y in zip(cpt_pct_l93.x_l93.tolist(), cpt_pct_l93.y_l93.tolist())]
db = DBSCAN(eps=200, min_samples=2).fit(data_test_clust)
labels = db.labels_
cpt_pct_l93['n_cluster']=labels

In [91]:
#recherche des nom_voie qui se ressemble dans le buffer
cpt_pct_l93['type_voie']=cpt_pct_l93.nom_voie.apply(lambda x : x.split(' ')[0])
cpt_pct_l93['suffix_nom_voie']=cpt_pct_l93.nom_voie.apply(lambda x : ' '.join(x.split(' ')[1:]).lower())
cross_join_ncluster=cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']].merge(
    cpt_pct_l93[['ident','nom_voie','type_voie','suffix_nom_voie', 'sens_circulation','annee','n_cluster']], on='n_cluster')
cross_join_ncluster['comp_nom_voie']=cross_join_ncluster.apply(
    lambda x : SequenceMatcher(None,x['suffix_nom_voie_x'], x['suffix_nom_voie_y']).ratio(), axis=1)

In [92]:
cross_join_ncluster.loc[(cross_join_ncluster['comp_nom_voie']>0.6)&
                        (cross_join_ncluster['nom_voie_x']!=cross_join_ncluster['nom_voie_y']) &
                        (cross_join_ncluster['n_cluster']!=-1)].sort_values(['n_cluster','ident_x'])

,ident_x,nom_voie_x,type_voie_x,suffix_nom_voie_x,sens_circulation_x,annee_x,n_cluster,ident_y,nom_voie_y,type_voie_y,suffix_nom_voie_y,sens_circulation_y,annee_y,comp_nom_voie
3446,728,rue de mahela,rue,de mahela,Sens unique vers la rue Fort Dauphin,2016,2,730,rue de majunca,rue,de majunca,Vers la rue du haut Brion,2016,0.631579
3447,728,rue de mahela,rue,de mahela,Sens unique vers la rue Fort Dauphin,2016,2,731,rue de majunca,rue,de majunca,Ves la rue de Tamatave,2016,0.631579
3459,729,rue de mahela,rue,de mahela,Sesn unique vers la rue du haut Brion,2016,2,730,rue de majunca,rue,de majunca,Vers la rue du haut Brion,2016,0.631579
3460,729,rue de mahela,rue,de mahela,Sesn unique vers la rue du haut Brion,2016,2,731,rue de majunca,rue,de majunca,Ves la rue de Tamatave,2016,0.631579
3470,730,rue de majunca,rue,de majunca,Vers la rue du haut Brion,2016,2,728,rue de mahela,rue,de mahela,Sens unique vers la rue Fort Dauphin,2016,0.631579
3471,730,rue de majunca,rue,de majunca,Vers la rue du haut Brion,2016,2,729,rue de mahela,rue,de mahela,Sesn unique vers la rue du haut Brion,2016,0.631579
3483,731,rue de majunca,rue,de majunca,Ves la rue de Tamatave,2016,2,728,rue de mahela,rue,de mahela,Sens unique vers la rue Fort Dauphin,2016,0.631579
3484,731,rue de majunca,rue,de majunca,Ves la rue de Tamatave,2016,2,729,rue de mahela,rue,de mahela,Sesn unique vers la rue du haut Brion,2016,0.631579
3919,590,rue henri iv,rue,henri iv,Sens unique vers la rue Lalande,2016,5,838,rue henri iv,rue,henri iv,Sens unique vers la rue de Lalande,2016,0.941176
3924,838,rue henri iv,rue,henri iv,Sens unique vers la rue de Lalande,2016,5,590,rue henri iv,rue,henri iv,Sens unique vers la rue Lalande,2016,0.941176


In [38]:
cpt_pct_l93.to_file(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_produites\Donnees\cluster_comptg_pctl.shp')

## 2. MISE EN FORME DU MMM

En premier lieu il faut mettre à jour les attributs en renomant de façon explicite puis en faisant la somme des 2 sens de circulation pour les voies doucble sens

In [4]:
#importer les données et convertir les noms de champs
fichier_src=gp.read_file(r'Q:\DAIT\TI\Bordeaux_metropole\2019\C19SA0101\Doc_travail\Donnees_produites\QGIS\tables\MMM_BdxM.shp')
#définir les colonnes définitives (PLUS TARD ON FERA UN FILTRE EN AMONT)
liste_col_def=['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 'VOLVEHPR~1', 
'Q_HC_PL', 'Q_HC_TV', 'Q_HC_VL', 'Q_HPM_PL', 'Q_HPM_TV', 'Q_HPM_VL', 'Q_HPS_PL', 'Q_HPS_TV', 'Q_HPS_VL', 'Q_Jour_PL', 'Q_Jour_TV',
'Q_Jour_VL','coef_HC_PL','coef_HC_TV','coef_HC_VL','Ocup_VL_PL','V0_PL','V0_VL','VCharg_PL','VCharg_VL','VOLPCUP~22',
'SHAREHGV', 'VOLVEH_~23', 'VOLVEH_~24', 'VOLCAPR~25', 'VEHHOUR~26', 'SPEEDLIMIT',
'R_NO', 'R_FROMNODENO', 'R_TONODENO', 'R_TYPENO', 'R_TSYSSET','R_LENGTH', 'R_NUMLANES', 'R_CAPPRT', 'r_V0_TV', 'R_VOLVEHPR~1', 
'R_Q_HC_PL', 'R_Q_HC_TV', 'R_Q_HC_VL', 'R_Q_HPM_PL', 'R_Q_HPM_TV', 'R_Q_HPM_VL', 
'R_Q_HPS_PL', 'R_Q_HPS_TV', 'R_Q_HPS_VL', 'R_Q_Jour_PL', 'R_Q_Jour_TV', 'R_Q_Jour_VL', 'R_coef_HC_PL', 'R_coef_HC_TV', 'R_coef_HC_VL', 'R_Ocup_VL_PL', 'R_V0_PL', 
'R_V0_VL', 'R_VCharg_PL', 'R_VCharg_VL', 'R_VOLPCUP~22', 'R_SHAREHGV', 'R_VOLVEH_~23', 'R_VOLVEH_~24', 'R_VOLCAPR~25', 'R_VEHHOUR~26', 'R_SPEEDLIMIT']
#creer un dico de renomage
dico_renomage={a: b for a, b in zip(fichier_src.columns,liste_col_def)}
#renommer
fichier_src.rename(columns=dico_renomage, inplace=True)

In [21]:
#ne conserver dans un premier temps que les attributs relatifs au trafic sur la journée
fichier_src_simpl=fichier_src[['NO', 'FROMNODENO', 'TONODENO', 'TYPENO', 'TSYSSET', 'LENGTH', 'NUMLANES', 'CAPPRT', 'V0_TV', 
             'VOLVEHPR~1']+[a for a in fichier_src.columns if 'Jour' in a ]].copy()

In [30]:
#caculer le tmja_TV
fichier_src_simpl['tmja_tv']=fichier_src_simpl.Q_Jour_TV+fichier_src_simpl.R_Q_Jour_TV

In [28]:
fichier_src_simpl.head(5)

,NO,FROMNODENO,TONODENO,TYPENO,TSYSSET,LENGTH,NUMLANES,CAPPRT,V0_TV,VOLVEHPR~1,Q_Jour_PL,Q_Jour_TV,Q_Jour_VL,R_Q_Jour_PL,R_Q_Jour_TV,R_Q_Jour_VL,tmja_tv
0,2000062,2000054,5868961,82,"A,B,BHNS,P,R,V",0.114km,1,600,40km/h,314,221.68,4422.38,4200.70,294.99,5945.08,5650.08,10367.46
1,2000063,2000054,6389027,82,"A,B,BHNS,P,R,V",0.099km,1,600,40km/h,467,294.99,5945.08,5650.08,221.68,4422.38,4200.70,10367.46
2,2000064,3527,2000055,77,"A,B,BHNS,P,R,V",0.160km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
3,2000065,2000055,4852848,77,"A,B,BHNS,P,R,V",0.037km,1,750,40km/h,269,582.93,9835.65,9190.25,424.10,6218.11,5755.06,16053.76
4,2000068,5743212,20000010,82,"A,B,BHNS,P,R,V",0.074km,1,600,40km/h,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00
